<a href="https://colab.research.google.com/github/dxda6216/q10/blob/main/circadian_period_q10_(median_temperature).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### This is a simple script to calculate Q10 values for circadian period
### length by using SciPy Optimize non-linear least squares fit on Colab.
### Copyright (c) 2022 by dxda6216 (dxda6216 AT gmail DOT com)
###
#@title Q10 calculator for circadian period
import numpy as np
import statistics
import pandas as pd
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt

#@markdown For usage, see [GitHub repository page](https://github.com/dxda6216/q10).

#@markdown Input data, then hit **Runtime** -> **Run all** (or press **Ctrl+F9**).

# Data description (plot title)
Data_description = "Mutant ABC" #@param {type:"string"}

# Temperature data
Temperatures = 25.6, 26.4, 26.8, 27.4, 27.6, 28.1, 28.6, 29.1, 29.6, 29.8, 30.5, 30.6, 30.8, 31, 31.1, 31.4, 31.6, 31.7, 31.8, 32, 32.4, 32.7, 33, 33.5, 33.8, 34.1, 34.3, 34.9, 35.4 #@param {type:"raw"}
x = np.array(Temperatures)

median_x = statistics.median(x)
maximum_x = max(x)
minimum_x = min(x)
average_x = statistics.mean(x)

# Period data
Periods =  23.1, 22.8, 22.5, 22, 22, 21.8, 21.2, 21, 20.7, 20.6, 20.6, 20.4, 20.4, 20.2, 20, 19.7, 19.9, 19.6, 19.7, 19.1, 19.2, 19.1, 18.9, 18.8, 18.7, 18.5, 18.1, 18.5, 17.9 #@param {type:"raw"}
y = np.array(Periods)

median_y = statistics.median(y)
maximum_y = max(y)
minimum_y = min(y)
average_y = statistics.mean(y)

# Pandas dataframe
tp = {'Temperature': x, 'Period': y}
df = pd.DataFrame(data=tp)
## Printing data
print(df)
print('\n')

# Base temperature
Select_base_temperature = "Median" #@param ["-273.15\u00B0C (absolute zero)", "0\u00B0C", "4\u00B0C", "25\u00B0C", "30\u00B0C", "37\u00B0C", "100\u00B0C", "Minimum", "Maximum", "Average", "Median", "Set \"Base_temperature\" by slider below"]
Base_temperature = 30 # @param {type:"slider", min:0, max:100, step:0.1}

if Select_base_temperature == "-273.15\u00B0C (absolute zero)":
	base_x = -273.15

if Select_base_temperature == "0\u00B0C":
	base_x = 0.000

if Select_base_temperature == "4\u00B0C":
	base_x = 4.000

if Select_base_temperature == "25\u00B0C":
	base_x = 25.000

if Select_base_temperature == "30\u00B0C":
	base_x = 30.000

if Select_base_temperature == "37\u00B0C":
	base_x = 37.000

if Select_base_temperature == "100\u00B0C":
	base_x = 100.000

if Select_base_temperature == "Minimum":
	base_x = minimum_x

if Select_base_temperature == "Maximum":
	base_x = maximum_x

if Select_base_temperature == "Average":
	base_x = average_x

if Select_base_temperature == "Median":
	base_x = median_x

if Select_base_temperature == "Set \"Base_temperature\" by slider below":
	base_x = Base_temperature

# Printing the max, min, etc. of the data
print(u'Minimum temperature =', '{:.3f}'.format(minimum_x), u'\u00B0C')
print(u'Maximum temperature =', '{:.3f}'.format(maximum_x), u'\u00B0C')
print(u'Average temperature =', '{:.3f}'.format(average_x), u'\u00B0C')
print(u'Median temperature  =', '{:.3f}'.format(median_x), u'\u00B0C\n')
print(u'Minimum period =', '{:.3f}'.format(minimum_y), u'h')
print(u'Maximum period =', '{:.3f}'.format(maximum_y), u'h')
print(u'Average period =', '{:.3f}'.format(average_y), u'h')
print(u'Median period  =', '{:.3f}'.format(median_y), u'h\n')

# Displaying Tab-delimited data Yes or No
Display_tab_delimited_data = "No" #@param ["Yes", "No"]

# Defining an equation for curve fitting
# fitting parameters:
#     tau_MT : period at median temperature (degree Celsius)
#     q10 : temperature coefficient (Q10)
def func(x, tau_MT, q10):
	return tau_MT / ( q10 ** ( ( x - base_x ) * 0.1 ) )

# Initial values for the fitting parameters
p0 = np.array([24.000, 1.000])

# Fitting the data to the defined equation
popt, pcov = curve_fit(func, x, y, p0)
residuals = y - func(x, *popt)
ss_residuals = np.sum(residuals**2)
ss_total = np.sum((y-np.mean(y))**2)
r_squared = 1 - ( ss_residuals / ss_total )

fig = plt.figure(figsize = (8,6))
fcxmin = int( min(x) - ( max(x) - min(x) ) * 0.333 )
fcxmax = int( max(x) + ( max(x) - min(x) ) * 0.333 ) + 1
fcx = np.linspace(fcxmin, fcxmax, 200)

plt.plot(x, y, 'o', color ='red', label ='data')
fcy = func(fcx, popt[0], popt[1])
plt.plot(fcx, fcy, '--', color='blue', label ='Fit    Q10 = %5.3f' %popt[1])
# plt.title(Data_description)
plt.xlabel(u'Temperature (\u00B0C)')
plt.ylabel('Period (hours)')

if Display_tab_delimited_data == "Yes":
	print(u'Dataset')
	print (u'Temp (\u00B0C)\tPeriod (hours)')
	ycount = 0
	for xseq in x:
		print(str(xseq)+'\t'+str(y[ycount]))
		ycount += 1
	print(u'\nFitted Curve')
	print (u'Temp (\u00B0C)\tPeriod (hours)')
	fcycount = 0
	for fcxseq in fcx:
		print('{:.3f}'.format(fcxseq)+'\t'+'{:.3f}'.format(fcy[fcycount]))
		fcycount += 1
	print(u'\n')

# Printing the results
print(u'Eestimated period length at', '{:.3f}'.format(base_x), u'\u00B0C =', '{:.3f}'.format(popt[0]), u'\u00B1', '{:.3f}'.format(pcov[0,0]**0.5), 'hours')
print(u'Q10 (temperature coefficient) =', '{:.3f}'.format(popt[1]), u'\u00B1', '{:.3f}'.format(pcov[1,1]**0.5))
print(u'R\u00B2 =', '{:.6f}'.format(r_squared), u'\n')

# Adjustment of plot location
plt.subplots_adjust(top=0.9, bottom=0.0)

# Fig title, header, footer
fig.suptitle(Data_description, fontsize=12)
fst = 'Eest. tau at ' + '{:.2f}'.format(base_x) + ' \u00B0C = ' + '{:.3f}'.format(popt[0]) + ' \u00B1 ' + '{:.3f}'.format(pcov[0,0]**0.5) + ' hrs      Q10 = ' + '{:.3f}'.format(popt[1]) + ' \u00B1 ' + '{:.3f}'.format(pcov[1,1]**0.5)
fig.text(0.5, 0.92, fst, horizontalalignment="center", fontsize=9)
# fig.text(0.5, 0.015, "text", horizontalalignment="center", fontsize=9)

plt.legend()
plt.show()

### End of script